# Balanced Random Cortex-like Network (Hard)
This task will create a network that this similar to part of the Cortex in the brain. This will take some input
from outside of the network, representing other surrounding neurons in the form of poisson spike sources.
These will then feed into an excitatory and an inhibitory network set up in a balanced random network. This
will use distributions of weights and delays as would occur in the brain.

Create a variable containing the number of neurons to be simulated in the network.

In [1]:
n_neurons = 100

Set up the simulation to use 0.1ms timesteps.

In [2]:
import pyNN.spiNNaker as p
p.setup(0.1)

Detected PyNN version 0.9.4 and Neo version 0.6.1


2021-04-10 14:27:06 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-10 14:27:06 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-10 14:27:06 WARNING: A timestep was entered that has forced sPyNNaker to automatically slow the simulation down from real time by a factor of 10. To remove this automatic behaviour, please enter a timescaleFactor value in your .spynnaker.cfg
2021-04-10 14:27:06 INFO: Setting time scale factor to 10.
2021-04-10 14:27:06 INFO: Setting machine time step to 100 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

Create an excitatory population with 80% of the neurons and an inhibitory population with 20% of
the neurons (hint: the number of neurons needs to be an integer).

In [3]:
exc = p.Population(int(n_neurons * 0.8), p.IF_curr_exp())
inh = p.Population(int(n_neurons * 0.2), p.IF_curr_exp())

Create excitatory poisson stimulation population with 80% of the neurons and an inhibitory poisson
stimulation population with 20% of the neurons, both with a rate of 1000Hz.


In [4]:
exc_poisson = p.Population(int(n_neurons * 0.8), p.SpikeSourcePoisson(rate=1000.0))
inh_poisson = p.Population(int(n_neurons * 0.2), p.SpikeSourcePoisson(rate=1000.0))

Create a one-to-one excitatory connection from the excitatory poisson stimulation population to the
excitatory population with a weight of 0.11nA and a delay of 1.0ms.

In [5]:
p.Projection(exc_poisson, exc, p.OneToOneConnector(), p.StaticSynapse(weight=0.11, delay=1.0), receptor_type="excitatory")

projection from pre SpikeSourcePoissonVertex_3 to post AbstractPopulationVertex_1 with connector OneToOneConnector()

Create a similar excitatory connection from the inhibitory poisson stimulation population to the
inhibitory population.

In [6]:
p.Projection(inh_poisson, inh, p.OneToOneConnector(), p.StaticSynapse(weight=0.11, delay=1.0), receptor_type="excitatory")

projection from pre SpikeSourcePoissonVertex_4 to post AbstractPopulationVertex_2 with connector OneToOneConnector()

Create two excitatory connections from the excitatory population to itself and to the inhibitory population. Use:
 - a fixed probability of connection of 0.1
 - a normal distribution of weights with a mean of 0.1 and a standard deviation of 0.1 (remember to add a boundary to make the weights positive)
 - a normal distribution of delays with a mean of 1.5 and standard deviation of 0.75 (remember to add aboundary to keep the delays within the allowed range of 0.1ms and 14.4ms on SpiNNaker).

In [7]:
import numpy
exc_weights = p.RandomDistribution("normal_clipped", mu=0.1, sigma=0.1, low=0, high=numpy.inf)
exc_delays = p.RandomDistribution("normal_clipped", mu=1.5, sigma=0.75, low=0.1, high=14.4)
p.Projection(exc, exc, p.FixedProbabilityConnector(0.1), p.StaticSynapse(weight=exc_weights, delay=exc_delays), receptor_type="excitatory")
p.Projection(exc, inh, p.FixedProbabilityConnector(0.1), p.StaticSynapse(weight=exc_weights, delay=exc_delays), receptor_type="excitatory")

projection from pre AbstractPopulationVertex_1 to post AbstractPopulationVertex_2 with connector FixedProbabilityConnector(0.1)

Create two similar connections between the inhibitory population and itself and the excitatory population, but make these "inhibitory" and make the weights four times as big.

In [8]:
inh_weights = p.RandomDistribution("normal_clipped", mu=0.4, sigma=0.1, low=0, high=numpy.inf)
inh_delays = p.RandomDistribution("normal_clipped", mu=1.5, sigma=0.75, low=0.1, high=14.4)
p.Projection(inh, inh, p.FixedProbabilityConnector(0.1), p.StaticSynapse(weight=inh_weights, delay=inh_delays), receptor_type="inhibitory")
p.Projection(inh, exc, p.FixedProbabilityConnector(0.1), p.StaticSynapse(weight=inh_weights, delay=inh_delays), receptor_type="inhibitory")

projection from pre AbstractPopulationVertex_2 to post AbstractPopulationVertex_1 with connector FixedProbabilityConnector(0.1)

Initialize the membrane voltages of the excitatory and inhibitory populations to a uniform random
number between -65.0 and -55.0.

In [9]:
exc.initialize(v=p.RandomDistribution("uniform", low=-65.0, high=-55.0))
inh.initialize(v=p.RandomDistribution("uniform", low=-65.0, high=-55.0))

Record the spikes from the excitatory population.

In [10]:
exc.record(["spikes", "v"])

Run the simulation for 1 second.

In [11]:
p.run(1000)

2021-04-10 14:27:11 INFO: Simulating for 10000 0.1ms timesteps using a hardware timestep of 10us
2021-04-10 14:27:11 INFO: Starting execution process
2021-04-10 14:27:14 INFO: Time 0:00:03.571219 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-10 14:27:24 INFO: Time 0:00:09.974834 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-10 14:27:30 INFO: Time 0:00:05.538399 taken by PartitionAndPlacePartitioner
Created spalloc job 5927596
2021-04-10 14:27:30 INFO: Created spalloc job 5927596
Waiting for board power commands to complete.
2021-04-10 14:27:30 INFO: Waiting for board power commands to complete.
2021-04-10 14:27:35 INFO: Time 0:00:05.052510 taken by SpallocAllocator
2021-04-10 1

1000.0

Retrieve and plot the spikes.  Ensure that you end the simulation!

In [12]:
spikes = exc.get_data("spikes")
voltage = exc.get_data('v')

p.end()

Getting spikes for AbstractPopulationVertex_1
|0%                          50%                         100%|
Getting v for AbstractPopulationVertex_1
|0%                          50%                         100%|


In [13]:
from pyNN.utility.plotting import Figure, Panel
Figure(Panel(spikes.segments[0].spiketrains))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The graph should show what is known as Asynchronous Irregular spiking activity - this means that the
neurons in the population don’t spike very often and when they do, it is not at the same time as other
neurons in the population.

## Extension

Try adding an additional Population of 100 neurons that stimulates the excitatory population with 50Hz noise using an all-to-all connector, and a weight of 0.01nA.  See what other behaviour you can get by trying to alter the rate of this new Population.

In [14]:
import matplotlib.pyplot as plt

def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            #print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

plt.matshow(voltage.segments[0].analogsignals[0].T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …